<a href="https://colab.research.google.com/github/Offliners/writeup/blob/main/HW7/homework7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [1]:
# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Fri May 21 04:12:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Download link 1
!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

!unzip -o hw7_data.zip

Downloading...
From: https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1
To: /content/hw7_data.zip
7.71MB [00:00, 5.72MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          


## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [3]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.2MB 7.4MB/s 
     |████████████████████████████████| 3.3MB 50.4MB/s 
     |████████████████████████████████| 901kB 44.7MB/s 


## Import Packages

In [4]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup
from random import randint

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [5]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 51kB 3.1MB/s 


## Load Model and Tokenizer




 

In [6]:
# model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  
tokenizer = AutoTokenizer.from_pretrained("wptoux/albert-chinese-large-qa")

model = AutoModelForQuestionAnswering.from_pretrained("wptoux/albert-chinese-large-qa")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [7]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [8]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [9]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 50
        self.max_paragraph_len = 300
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 128

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start

            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
# temp_set = ConcatDataset([train_set, dev_set])
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [84]:
def evaluate(data, output, index):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])

            if answer.find('[UNK]') != -1:
                print(f'ID {index} Before : {answer}')
                start = start_index - min(len(test_questions[index]['question_text']), 50) - 2 + k * 128
                end = end_index - min(len(test_questions[index]['question_text']), 50) - 2 + k * 128
                start_target = test_paragraphs_tokenized[test_questions[index]['paragraph_id']].offsets[start]
                end_target = test_paragraphs_tokenized[test_questions[index]['paragraph_id']].offsets[end]
                target = test_paragraphs[test_questions[index]['paragraph_id']][start_target[0]: end_target[1]]
                answer = target
                print(f'ID {index} After : {answer}')

    answer = answer.replace('[CLS]','')                   
    answer = answer.replace('[SEP]','')
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

## Training

In [11]:
num_epoch = 3
validation = True
logging_step = 100
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * num_epoch
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=total_steps)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

accum_iter = 4 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0

    for batch_idx, data in enumerate(tqdm(train_loader)):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        with torch.set_grad_enabled(True):
            # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
            # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
            output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

            # Choose the most probable start position / end position
            start_index = torch.argmax(output.start_logits, dim=1)
            end_index = torch.argmax(output.end_logits, dim=1)
            
            # Prediction is correct only if both start_index and end_index are correct
            train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
            train_loss += output.loss
            
            output.loss = output.loss / accum_iter

            if fp16_training:
                accelerator.backward(output.loss)
            else:
                output.loss.backward()
            
            if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
                optimizer.step()
                optimizer.zero_grad()
            
            step += 1

            ##### TODO: Apply linear learning rate decay #####
            scheduler.step()

            # Print training loss and accuracy over past logging step
            if step % logging_step == 0:
                print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
                train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, i) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 1 | Step 100 | loss = 2.278, acc = 0.315
Epoch 1 | Step 200 | loss = 1.691, acc = 0.462
Epoch 1 | Step 300 | loss = 1.154, acc = 0.567
Epoch 1 | Step 400 | loss = 0.843, acc = 0.676
Epoch 1 | Step 500 | loss = 0.834, acc = 0.700
Epoch 1 | Step 600 | loss = 0.715, acc = 0.732
Epoch 1 | Step 700 | loss = 0.698, acc = 0.724
Epoch 1 | Step 800 | loss = 0.635, acc = 0.731
Epoch 1 | Step 900 | loss = 0.664, acc = 0.735
Epoch 1 | Step 1000 | loss = 0.624, acc = 0.744
Epoch 1 | Step 1100 | loss = 0.661, acc = 0.735
Epoch 1 | Step 1200 | loss = 0.615, acc = 0.748
Epoch 1 | Step 1300 | loss = 0.590, acc = 0.746
Epoch 1 | Step 1400 | loss = 0.599, acc = 0.762
Epoch 1 | Step 1500 | loss = 0.618, acc = 0.739
Epoch 1 | Step 1600 | loss = 0.621, acc = 0.781
Epoch 1 | Step 1700 | loss = 0.655, acc = 0.730
Epoch 1 | Step 1800 | loss = 0.540, acc = 0.776
Epoch 1 | Step 1900 | loss = 0.557, acc = 0.777
Epoch 1 | Step 2000 | loss = 0.516, acc = 0.799
Epoch 1 | Step 2100 | loss = 0.532, acc = 0.776
E

ID 140 Before : 單 挺 機 槍 的 理 論 循 環 射 速 較 高 ， 同 樣 的 射 擊 時 間 內 能 夠 投 射 較 多 的 彈 頭 數 量 ， 總 合 重 量 不 低 。 可 是 缺 點 是 彈 頭 本 身 的 威 力 較 低 ， 面 對 一 些 保 護 設 計 甚 至 完 全 無 法 貫 穿 ， 比 如 蘇 聯 著 名 的 重 裝 甲 攻 擊 機 [UNK] - 2 和 美 國 的 重 型 轟 炸 機 。 採 用 少 量 的 機 砲 ， 提 升 每 一 門 的 破 壞 力 ， 但 是 當 時 機 砲 的 循 環 射 速
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 408 Before : 《 藍 血 人 》 等 。 倪 匡 作 品 被 翻 拍 成 多 部 香 港 電 影 及 電 視 劇 集 。 但 是 ， 也 有 反 對 倪 匡 的 人 認 為 倪 匡 的 作 品 不 是 科 幻 ， 只 能 成 為 奇 幻 或 是 偽 科 幻 。 83 [UNK] 84 年 間 ， 中 國 科 幻 文 學 被 貼 上 精 神 污 染 的 標 籤 ， 受 到 嚴 厲 懲 處 的 事 件 ， 這 次 抵 制 精 神 污 染 的 政 治 運 動 幾 乎 使 整 個 中 國 科 幻 事 業 夭 折 。 運 動 產 生 的 經 濟 和 文 化 後 果 則 無 法 估 量 。 再 如 ， 主 流 科 幻 作 家 中 ， 鄭 文 光 因 此 一 病 不 起 ， 葉 永 烈 、 童 恩 正 、 劉 興 詩 、 肖 建 亨 等 受 到 不 實 污 [UNK] 和 指 控 。 1978 年 在 大 陸 創 辦 的 《
Target : 年
ID 408 After  : 《 藍 血 人 》 等 。 倪 匡 作 品 被 翻 拍 成 多 部 香 港 電 影 及 電 視 劇 集 。 但 是 ， 也 有 反 對 倪 匡 的 人 認 為 倪 匡 的 作 品 不 是 科 幻 ， 只 能 成 為 奇 幻 或 是 偽 科 幻 。 83 年 84 年 間 ， 中 國 科 幻 文 學 被 貼 上 

Epoch 2 | Step 100 | loss = 0.402, acc = 0.810
Epoch 2 | Step 200 | loss = 0.391, acc = 0.814
Epoch 2 | Step 300 | loss = 0.446, acc = 0.822
Epoch 2 | Step 400 | loss = 0.446, acc = 0.804
Epoch 2 | Step 500 | loss = 0.439, acc = 0.805
Epoch 2 | Step 600 | loss = 0.423, acc = 0.811
Epoch 2 | Step 700 | loss = 0.407, acc = 0.815
Epoch 2 | Step 800 | loss = 0.434, acc = 0.812
Epoch 2 | Step 900 | loss = 0.379, acc = 0.824
Epoch 2 | Step 1000 | loss = 0.383, acc = 0.826
Epoch 2 | Step 1100 | loss = 0.457, acc = 0.799
Epoch 2 | Step 1200 | loss = 0.376, acc = 0.831
Epoch 2 | Step 1300 | loss = 0.401, acc = 0.829
Epoch 2 | Step 1400 | loss = 0.412, acc = 0.819
Epoch 2 | Step 1500 | loss = 0.419, acc = 0.809
Epoch 2 | Step 1600 | loss = 0.402, acc = 0.829
Epoch 2 | Step 1700 | loss = 0.378, acc = 0.834
Epoch 2 | Step 1800 | loss = 0.413, acc = 0.796
Epoch 2 | Step 1900 | loss = 0.407, acc = 0.817
Epoch 2 | Step 2000 | loss = 0.372, acc = 0.825
Epoch 2 | Step 2100 | loss = 0.388, acc = 0.822
E

ID 32 Before : 周 王 是 燕 王 同 母 兄 弟 ， 而 朱 允 [UNK] 怕 他 與 燕 王 呵 成 一 氣
ID 70 Before : 1998 年 推 出 了 163. com 為 域 名 的 免 費 [UNK] 郵 件 服 務 ， 163 是 當 時 中 國 電 信 最 常 見 的 撥 號 上 網 的 號 碼 ， 普 遍 認 為 這 一 通 俗 易 記 的 名 稱 對 網 易 [UNK] 服 務 的 成 功 起 到 了 重 要 的 作 用 。 後 來 網 易 跟 隨 新 浪 和 搜 狐 ， 推 出 了 入 口 網 站 服 務 ， 經 常 與 後 兩 者 並 稱 為 三 大 門 戶 。 網 易 於 2000 年
ID 134 Before : 基 岩 裸 露 、 土 層 較 薄 ， 但 是 島 嶼 內 仍 然 有 溪 流 提 供 淡 水 ， 而 附 近 海 域 也 有 大 量 魚 群 棲 息 著 。 其 中 為 了 適 應 釣 魚 臺 位 於 海 上 強 風 的 自 然 環 境 ， 許 多 動 植 物 都 各 自 發 展 成 為 特 有 種 ， 這 包 括 有 海 芙 蓉 、 釣 魚 臺 [UNK] 、 釣 魚 島 菠 蘿 、 黃 尾 嶼 龍 蝦 、 黃 尾 嶼 蜈 蚣 、 釣 魚 島 細 辛 、 尖 閣 葵 、 尖 閣 弟 切 草 等 動 植 物 。 由 於 島 上 淡 水 資 源 缺 乏 且 無 長 期 存 在 的 淡 水 水 體
Target : 
ID 134 After  : 基 岩 裸 露 、 土 層 較 薄 ， 但 是 島 嶼 內 仍 然 有 溪 流 提 供 淡 水 ， 而 附 近 海 域 也 有 大 量 魚 群 棲 息 著 。 其 中 為 了 適 應 釣 魚 臺 位 於 海 上 強 風 的 自 然 環 境 ， 許 多 動 植 物 都 各 自 發 展 成 為 特 有 種 ， 這 包 括 有 海 芙 蓉 、 釣 魚 臺  、 釣 魚 島 菠 蘿 、 黃 尾 嶼 龍 蝦 、 黃 尾 嶼 蜈 蚣 、 釣 魚 島 細 辛 、 尖 閣 葵 、 尖 閣 弟 切 草 等 動 植 物 。 由 於 島 上 淡 水 資 源 缺 乏 且 無 長 期 存 在 的 淡 水 水 體
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 

Epoch 3 | Step 100 | loss = 0.293, acc = 0.860
Epoch 3 | Step 200 | loss = 0.329, acc = 0.852
Epoch 3 | Step 300 | loss = 0.276, acc = 0.870
Epoch 3 | Step 400 | loss = 0.308, acc = 0.859
Epoch 3 | Step 500 | loss = 0.323, acc = 0.851
Epoch 3 | Step 600 | loss = 0.336, acc = 0.854
Epoch 3 | Step 700 | loss = 0.334, acc = 0.861
Epoch 3 | Step 800 | loss = 0.339, acc = 0.839
Epoch 3 | Step 900 | loss = 0.330, acc = 0.831
Epoch 3 | Step 1000 | loss = 0.360, acc = 0.829
Epoch 3 | Step 1100 | loss = 0.327, acc = 0.860
Epoch 3 | Step 1200 | loss = 0.325, acc = 0.846
Epoch 3 | Step 1300 | loss = 0.354, acc = 0.842
Epoch 3 | Step 1400 | loss = 0.272, acc = 0.887
Epoch 3 | Step 1500 | loss = 0.294, acc = 0.865
Epoch 3 | Step 1600 | loss = 0.364, acc = 0.857
Epoch 3 | Step 1700 | loss = 0.290, acc = 0.846
Epoch 3 | Step 1800 | loss = 0.340, acc = 0.835
Epoch 3 | Step 1900 | loss = 0.351, acc = 0.832
Epoch 3 | Step 2000 | loss = 0.284, acc = 0.851
Epoch 3 | Step 2100 | loss = 0.341, acc = 0.854
E

ID 32 Before : 周 王 是 燕 王 同 母 兄 弟 ， 而 朱 允 [UNK] 怕 他 與 燕 王 呵 成 一 氣
ID 70 Before : 1998 年 推 出 了 163. com 為 域 名 的 免 費 [UNK] 郵 件 服 務 ， 163 是 當 時 中 國 電 信 最 常 見 的 撥 號 上 網 的 號 碼 ， 普 遍 認 為 這 一 通 俗 易 記 的 名 稱 對 網 易 [UNK] 服 務 的 成 功 起 到 了 重 要 的 作 用 。 後 來 網 易 跟 隨 新 浪 和 搜 狐 ， 推 出 了 入 口 網 站 服 務 ， 經 常 與 後 兩 者 並 稱 為 三 大 門 戶 。 網 易 於 2000 年 在 美 國 納 斯 達 克 上 市 ， 2001 年
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 414 Before : [UNK] 大 壩
Target After: 法規
ID 414 After  : 法規 大 壩
ID 529 Before : 《 阿 [UNK] 婆 吠 陀 》
ID 716 Before : 姚 [UNK]
ID 811 Before : 蘇 聯 [UNK] - 47 突 擊 步 槍 的 中 方 授 權 製 造 版 本 56 式 自 動 步 槍
ID 826 Before : 成 為 南 安 縣 二 十 位 [UNK] 膳 生 之 一 。 崇 禎 十 四 年 ， 迎 娶 福 建 泉 州 惠 安 進 士 禮 部 侍 郎 董 [UNK] 先 侄 女 董 友 。 崇 禎 十 七 年
ID 854 Before : [UNK]
ID 883 Before : [UNK] 水 之 戰
ID 924 Before : [UNK] [UNK] 永 遠 懷 念
ID 937 Before : 長 [UNK] 米
ID 961 Before : 基 金 [UNK] [UNK] 包 括 中 國 紅 十 字 會 在 內 的 公 募 基 金
ID 1029 Before : 野 生 動 物 ， 另 一 個 是? [SEP] 阿 

In [12]:
test_paragraphs[test_questions[49]['paragraph_id']]

'部份香港政府機關在九龍灣設置總部，包括機電工程署總部大樓、香港輔助警察隊總部，郵政署亦於2013年在該處設立中央郵件中心。附近亦有九龍灣運動場、公園、體育館等公共設施。九龍灣國際展貿中心是香港主要的國際商品展覽會場之一，附設的辦公室亦可作不同產品陳列室之用。現時國際展貿中心仍繼續舉辦不同的國際展覽，亦會不時舉辦演唱會。2007年，自大型購物中心MegaBox開幕後，改變了九龍灣作為工廠、物流倉庫的印象。MegaBox商場高18層，佔地達110萬平方呎，是觀塘區最大型的購物商場，當中的大型商戶有宜家傢俬、AEON、IMAX影院等。區內的寫字樓自2006年起不斷增建，包括企業廣場三期、國際交易中心、Manhattan Place、傲騰廣場及億京中心等，成為東九龍新商業中心。2012年，香港首座零碳建築零碳天地落成，為香港政府宣布《起動九龍東》計劃後，成為九龍東首批標誌性項目之一。九龍灣也有一些住宅區，有兩大私人屋苑，分別是九龍灣站上蓋的德福花園及麗晶花園。'

In [23]:
# Postprocessing

def postprocessing(result, index):
    if result:
        if result[0] == '，' or result[0] == '。':
            print(f'ID {index} Before : {result}')
            result = result[1:]
            print(f'ID {index} After  : {result}')

        if  result.find('《') != -1 and result.find('》') == -1:
            print(f'ID {index} Before : {result}')
            result = result + '》'
            print(f'ID {index} After  : {result}')
        
        if result.find('《') == -1 and result.find('》') != -1:
            print(f'ID {index} Before : {result}')
            result = '《' + result  
            print(f'ID {index} After  : {result}')

        if  result.find('「') != -1 and result.find('」') == -1:
            print(f'ID {index} Before : {result}')
            result = result + '」'
            print(f'ID {index} After  : {result}')
        
        if result.find('「') == -1 and result.find('」') != -1:
            print(f'ID {index} Before : {result}')
            result = '「' + result  
            print(f'ID {index} After  : {result}')
        
        
    return result

## Testing

In [85]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, i))

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
                result[i] = postprocessing(result[i], i)
                f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


ID 26 Before : 拉 丁 文 [UNK]
ID 26 After : 拉丁文Civilis
ID 33 Before : 爪 哇 。 元 代 航 海 家 汪 大 淵 《 島 夷 誌 略 》 記 載 「 石 塘 之 骨 ， 由 潮 州 而 生 ， 迤 [UNK] 如 長 蛇 ， 橫 海 中 ， 越 海 諸 國 。 俗 雲 萬 里 石 塘 」 。 明 代 永 樂 年 間 的 隨 鄭 和 下 西 洋
ID 33 After : 爪哇。元代航海家汪大淵《島夷誌略》記載「石塘之骨，由潮州而生，迤邐如長蛇，橫海中，越海諸國。俗雲萬里石塘」。明代永樂年間的隨鄭和下西洋
ID 49 Before : 大 型 購 物 中 心 [UNK] 開 幕
ID 49 After : 大型購物中心MegaBox開幕
ID 59 Before : 誤 將 「 [UNK] [UNK]
ID 59 After : 誤將「Hu Jintao
ID 84 Before : 平 息 下 來 ， 當 勝 利 鐘 的 照 片 刊 在 [UNK] 雜 誌 " [UNK] [UNK], " 上 時 ， 引 起 了 軒 然 大 波 ， [UNK] 的 學 生 朝 [UNK] 的 特 洛 依 銅 像 [UNK] [UNK] 噴 漆 ， 而 [UNK] 的 學 生 也 不 甘 示 弱 的 回 敬 在 [UNK] 的 草 皮 上 燒 出 [UNK] 的 縮 寫 ， 讓 警 方 不 得 不 數 次 警 告 ， 最 後 ， 這 場 衝 突 在 [UNK] 校 長 [UNK]. [UNK] [UNK]. von [UNK]
ID 84 After : 平息下來，當勝利鐘的照片刊在USC雜誌"The Wampus,"上時，引起了軒然大波，UCLA的學生朝USC的特洛依銅像Tommy Trojan噴漆，而USC的學生也不甘示弱的回敬在UCLA的草皮上燒出USC的縮寫，讓警方不得不數次警告，最後，這場衝突在USC校長Dr. Rufus B. von Kleinsmid
ID 84 Before : [UNK]
ID 84 After : USC
ID 84 Before : 小 熊 ， 因 為 他 們 從 未 打 敗 過 [UNK] ， 但 在 第 一 年 的 比 賽 ， 1942 年 ， [UNK]
ID 84 Afte

In [86]:
from google.colab import files
files.download("result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>